# 大模型并发调用

## 文本生成
并行/串行调用10次大模型，对比调用时间

In [ ]:
# 在notebook环境中运行，需要更改系统路径
import sys
import os
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

In [11]:
import asyncio
import time

from langchain.chains.llm import LLMChain
from langchain_core.prompts import PromptTemplate

from src.llm.llm_factory import LLMFactory

In [12]:
prompt_template = "给我讲个50字以内的笑话"
prompt = PromptTemplate(
    input_variables=[],
    template=prompt_template
)

def generate_serially():
    llm = LLMFactory.get_openai_factory().get_chat_llm_3()
    chain = LLMChain(
        llm=llm,
        prompt=prompt,
    )
    for _ in range(10):
        content = chain.invoke(input={})['text']
        print(content)


async def async_generate(chain):
    resp = await chain.arun(input={}) # 最好该用ainvoke
    print(resp)


async def generate_concurrently():
    llm = LLMFactory.get_openai_factory().get_chat_llm_3()
    chain = LLMChain(
        llm=llm,
        prompt=prompt,
    )
    tasks = [async_generate(chain) for _ in range(10)]
    await asyncio.gather(*tasks)

并行

In [18]:
async def main():
    s = time.perf_counter()
    await generate_concurrently()
    elapsed = time.perf_counter() - s
    print("\033[1m" + f"Concurrent executed in {elapsed:0.2f} seconds." + "\033[0m")

await main() # 在.py文件中，改为asyncio.run(main())

为什么猫咪不喜欢下雨天？因为它们怕变成“湿猫”！
为什么学校的操场是圆的？
因为老师说：“让学生四处跑！”
为什么猫咪不喜欢下雨天？
因为它们怕自己会变成“湿猫”！
为什么太阳总是这么生气？因为它总是起床都没吃早饭就开始工作了！
为什么猫咪喜欢在键盘上踩来踩去？因为它们想成为“鼠标”专家！
为什么猫咪不喜欢打扑克？
因为它们总是被抓住！
为什么猫咪不喜欢下雨天？
因为它们怕自己会变成“湿猫”！😸
为什么小明的作业总是写不完呢？因为他总是在作业本上写着写着就睡着了。
为什么猫咪喜欢玩纸箱？
因为它们觉得纸箱是“盒”乐无穷！
为什么学校的操场很干净？因为同学们都把垃圾扔到了教室里。
Concurrent executed in 4.35 seconds.


串行

In [20]:
s = time.perf_counter()
generate_serially()
elapsed = time.perf_counter() - s
print("\033[1m" + f"Serial executed in {elapsed:0.2f} seconds." + "\033[0m")

为什么学校的操场是圆的？因为校长说：学生都是圆滚滚的，操场也要跟着他们一起圆滚滚。
为什么月亮喜欢穿新衣服？因为它每个月都要换一次！
为什么手机喜欢在夜里响个不停？因为它是“夜猫子”！
为什么蘑菇是个好朋友？因为它总是"菇"着你！
为什么蜜蜂从来不会迟到？因为它们总是在蜜时间到达！
为什么小明不敢去参加跳水比赛？

因为他怕跳水后会变成小明泡泡！
为什么蘑菇是个好朋友？因为它总是“菇”得你开心！
为什么猫咪不喜欢下雨天？因为它们怕变成“湿猫”！
为什么猫咪喜欢玩扑克牌？因为它们总是抓得住鱼！
为什么学校的操场很干净？
因为学生们都把垃圾扔到了教室里。
Serial executed in 9.83 seconds.


## 图像理解
对于同一张图像，并行/串行调用3次，对比调用时间

In [22]:
import asyncio
import base64
from langchain_core.messages import HumanMessage
from src.llm.llm_factory import LLMFactory

In [28]:
image_path = "D:\\download\\case\\images\\客服数字人直播增强互动性讲解需求描述_7-36.png"
prompt = "请详细描述图像的内容"

llm = LLMFactory.get_openai_factory().get_gpt4o()

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")
        

并行

In [31]:
async def analyze_image_concurrently(image_path, prompt):
    base64_image = encode_image(image_path)
    message = HumanMessage(
        content=[
            {"type": "text", "text": prompt},
            {
                "type": "image_url",
                "image_url": {"url": f"data:image/png;base64,{base64_image}"}
            }
        ]
    )
    output = await llm.ainvoke([message])
    print(output.content)

async def image_tasks():
    tasks = [analyze_image_concurrently(image_path, prompt) for _ in range(3)]
    await asyncio.gather(*tasks) 
       
async def main():
    s = time.perf_counter()
    await image_tasks()
    elapsed = time.perf_counter() - s
    print("\033[1m" + f"Concurrent executed in {elapsed:0.2f} seconds." + "\033[0m")

await main() # 在.py文件中，改为asyncio.run(main())

这是一张显示文本编辑器界面的图像。顶部有一排工具栏，包括以下图标：

1. `</>` - 代码视图图标
2. `B` - 加粗图标
3. `I` - 斜体图标
4. `S` - 删除线图标
5. 铅笔图标 - 编辑图标
6. 画笔图标 - 绘图工具图标
7. `T` - 大写字体图标
8. 链接图标 - 插入链接图标

在工具栏下方的编辑区域，有一行蓝色字体的中文文字：“请戳这里”。
这是一张截屏图像，显示了一个文本编辑器的界面。该界面的上方有一排工具栏，包含了各种文本格式化工具。具体工具从左到右依次是：

1. 代码插入工具（显示为一个尖括号符号）
2. 粗体（B）
3. 斜体（I）
4. 下划线（U）
5. 删除线（S）
6. 链接插入工具（一个链条符号）
7. 笔刷工具（用于清除格式）
8. 调色板工具（用于更改文本颜色）
9. 段落对齐工具（用于调整文本对齐方式）
10. 列表工具（用于创建有序或无序列表）

在文本编辑区中，有一行蓝色的文字，内容是“请戳这里”。
该图像显示了一个文本编辑器的界面，类似于电子邮件或网页内容管理系统中的编辑器。上方有一排工具栏，包含各种文本编辑工具，从左到右依次为：

1. HTML代码编辑按钮（图标是一个尖括号）
2. 粗体按钮（B）
3. 斜体按钮（I）
4. 删除线按钮（S）
5. 链接按钮（图标像一个链条）
6. 取消链接按钮（图标像一个断开的链条）
7. 插入或编辑图片按钮（图标是一个山的图案）
8. 插入或编辑视频按钮（图标是一个视频摄像机）
9. 插入或编辑表格按钮（图标是一个表格）
10. 有序列表按钮（图标是一个带数字的列表）
11. 无序列表按钮（图标是一个带点的列表）

在工具栏下方的编辑区域里，有一行蓝色的中文文字，写着“请戳这里”。
Concurrent executed in 10.88 seconds.


串行

In [29]:
def analyze_image_serially(image_path, prompt):
    base64_image = encode_image(image_path)
    message = HumanMessage(
        content=[
            {"type": "text", "text": prompt},
            {
                "type": "image_url",
                "image_url": {"url": f"data:image/png;base64,{base64_image}"}
            }
        ]
    )
    return (llm.invoke([message]))

s = time.perf_counter()
for _ in range(3):
    response = analyze_image_serially(image_path, prompt)
    print(response.content)
elapsed = time.perf_counter() - s
print("\033[1m" + f"Serial executed in {elapsed:0.2f} seconds." + "\033[0m")
    

这是一张显示文本编辑器界面的截图。编辑器的工具栏位于顶部，其中包含以下功能按钮，从左到右依次是：

1. 代码（<>）按钮
2. 加粗（B）按钮
3. 斜体（I）按钮
4. 划线（S）按钮
5. 下划线（U）按钮
6. 字体颜色（铅笔图标）按钮
7. 背景颜色（画笔图标）按钮
8. 字体大小（Tt）按钮
9. 链接（链条图标）按钮

工具栏下面是一个文本输入区域，里面有一行蓝色的文字，内容是“请戳这里”。
这是一张显示文本编辑器界面的图片。界面上方有一排工具栏，包括多种文本编辑工具，从左到右依次为：

1. 代码查看按钮（显示为 `<>` 符号）
2. 加粗按钮（显示为字母B）
3. 斜体按钮（显示为字母I）
4. 下划线按钮（显示为字母U）
5. 删除线按钮（显示为字母S和一条横线）
6. 链接按钮（显示为一个链条的图标）
7. 橡皮擦按钮（用于清除格式）
8. 调整文本颜色和背景色的按钮（显示为铅笔和画笔的图标）
9. 字体大小调整按钮（显示为字母T和一个小三角形）
10. 列表按钮（显示为三条横线和小圆点，表示项目符号列表）

在文本编辑区域内有一行蓝色文字，内容为“请戳这里”。
这是一张截图，显示了一个文本编辑器的界面。界面顶部有一行工具栏，包含了一些常见的文本编辑选项，从左到右依次是：

1. 代码按钮：用于插入代码片段。
2. 粗体按钮：用于将选中的文本设置为粗体。
3. 斜体按钮：用于将选中的文本设置为斜体。
4. 下划线按钮：用于为选中的文本添加下划线。
5. 删除线按钮：用于在选中的文本上添加删除线。
6. 铅笔按钮：用于编辑文本。
7. 铅笔图标旁的按钮：可能是用于更改文本颜色或背景颜色的工具。
8. 字体按钮：用于更改文本的字体。
9. 无序列表按钮：用于创建无序列表。
10. 有序列表按钮：用于创建有序列表。
11. 引用按钮：用于插入引用文本。
12. 链接按钮：用于插入超链接。

在工具栏下方的编辑区域中，有一行文字，内容为“请戳这里”，文字为蓝色，可能是一个超链接。
Serial executed in 33.28 seconds.
